# Collect energy statistics from D-Wave

In this notebook you will find some code useful to replicate the experiments on D-Wave in the paper (referenced in the README). In particular there is some code to sample the initial sping configurations at a selected temperature and then the code to run the reverse annealing experiments on the D-Wave hardware.

To make use of this second part you will nead a Leap accunt with 1 minute of QPU time available (with the current parameters you will use roughly 90% of that minute).

For further informations please contact the authors.

In [ ]:
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import pickle
import time
import matplotlib.pyplot as plt
import pandas as pd
import dimod
from dimod.reference import samplers
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite, FixedEmbeddingComposite
from minorminer import find_embedding

In [ ]:
class Ising():
    ''' Simulating the 2D Ising model '''
    def __init__(self, N, temp, J = 1, h = 0, config = None):
        self.N = N
        self.T = temp
        self.J = J
        self.h = h
        self.time = 0
        self.config = config


    def energy(self):
        '''Energy of a given configuration'''
        energy = 0
        for i in range(len(self.config)):
            S = self.config[i]
            nb = self.config[(i+1)%self.N] + self.config[(i-1)%self.N] 
            energy += self.J*nb*S + self.h*S
        return energy/self.N


    def magnetization(self):
        '''Magnetization of a given configuration'''
        mag = np.sum(self.config)
        return mag


    ## monte carlo moves
    def mcmove(self):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        beta = 1.0/self.T
        for i in range(self.N):          
            a = np.random.randint(0, self.N)
            s =  self.config[a]
            nb = self.config[(a+1)%self.N] + self.config[(a-1)%self.N] 
            cost = -2*self.J*s*nb - 2*self.h*s 
            if cost < 0 or rand() < np.exp(-cost*beta):
                s *= -1
            self.config[a] = s


    def evolve(self, n_iter, h_values=None):   
        ''' This module simulates the evolution of Ising model'''
        for i in range(n_iter):
            self.mcmove()
        spin_conf = self.config
        return spin_conf, self.energy()


    def thermalize(self):
        equilibration = 1
        while equilibration > 0.5: 
            self.mcmove()
            self.time += 1
            if self.time%500 == 0:
                equilibration = 0


    def sample_statistics(self, n_iter=1000):
        #Initial state is thermalized
        initial_configurations = []
        initial_energies = []
        for i in range(10):
            self.config = 2*np.random.randint(2, size=(self.N))-1
            self.thermalize()
            for i in range(int(n_iter/10)):
                config, en = self.evolve(100)
                initial_configurations.append(config)
                initial_energies.append(en)
        return initial_configurations, initial_energies

In [ ]:
#Global variables and problem setting
chain_lenght = 300
num_samples = 3000

h = dict(enumerate([0.0 for i in range(chain_lenght-1)]))
J = {(i, i+1): 1.0 for i in range(chain_lenght-1)}
topology = [(i, i+1) for i in range(chain_lenght-1)]

bqm = dimod.BinaryQuadraticModel.from_ising(h, J)
qpu_sampler = DWaveSampler(solver='DW_2000Q_6') #DW_2000Q_2_1 or DW_2000Q_5(lower noise)
embedding = {}
while embedding == {}:
    embedding = find_embedding(topology,qpu_sampler.edgelist)

sampler = FixedEmbeddingComposite(qpu_sampler, embedding)

H = -(-np.diag(np.ones(chain_lenght-1), -1) -np.diag(np.ones(chain_lenght-1), 1))/chain_lenght

print('Found a valid embedding for the specified problem.')

In [ ]:
#Important parameters!
T = 0.5
s_bar =0.3
 
N = chain_lenght
initial_config = 2*np.random.randint(2, size=(N))-1
rm = Ising(N, T, J=1, h=0, config=initial_config)
initial_configurations, initial_energies = rm.sample_statistics(num_samples)
print('Sampled initial states from thermal distribution.')

In [ ]:
#Work statistics at different anneal_param
anneal_lenght = 2

initial_energy_stat = []
final_energy_stat = []

tic = time.time()
for anneal_param in [s_bar]:
    E_in = []
    E_fin = []
    for i in range(num_samples):
        #init_state = np.random.choice([-1,1],size=(chain_lenght,),p=[1/2,1/2])
        init_state = initial_configurations[i]
        E_init = np.dot(init_state,np.dot(H,init_state))
        initial_state= dict(enumerate(init_state.tolist()))

        samples = sampler.sample(bqm, initial_state=initial_state, anneal_schedule=[[0,1], [anneal_lenght/2,anneal_param], [anneal_lenght,1]], num_reads=10)
        for s in samples.samples():
            final_state = np.array(list(s.values()))
            E_in.append(E_init)
            E_fin.append(np.dot(final_state,np.dot(H,final_state)))
    initial_energy_stat.append(E_in)
    final_energy_stat.append(E_fin)
    with open('DAR_T05_s03.pkl', 'wb') as f:
        pickle.dump([initial_energy_stat, final_energy_stat], f)    
toc = time.time() 

print('Collected proper work statistics.')
print('Elapsed time:', toc-tic,'s.\n')

In [ ]:
with open('DAR_T05_s03.pkl', 'rb') as f:
    initial_energy_stat, final_energy_stat = pickle.load(f)

In [ ]:
delta_e = (np.array(final_energy_stat)-np.array(initial_energy_stat)).tolist()
plt.hist(delta_e, bins='doane')
plt.ylabel('counts')
plt.xlabel('$\Delta E$')
plt.show()